# MIDI.ME

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_context('talk',font_scale=.8)
import sys, os, os.path
from scipy.io import wavfile
from scipy import signal
from mido import MidiFile

chops = []
bps = np.array([])

mid = MidiFile('200415a.mid')
print(mid.ticks_per_beat)
ticks_per_second=mid.ticks_per_beat*2
ticks_per_second=210
time = 0
beat = 0

for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
        if msg.type == 'note_off':
            beat += (msg.time / ticks_per_second)
            time += (msg.time / ticks_per_second)
            print ("time: " + str(msg.time))
          #  print (beat)
           # print (time)
        elif msg.type == 'note_on':
            beat += (msg.time / ticks_per_second)
            time += (msg.time / ticks_per_second)
            chops.append(time)
            bps = np.append(bps, 1/beat)
            beat = 0

print(time)
input_filename = '200415a_test.wav'
samrate, data = wavfile.read(str(input_filename))
print (len(data)/samrate)
print(np.sum(1/bps))
print(bps.size)

print("chops " + str(chops))


bps = 1/(bps*(np.sum(1/bps)/(len(data)/samrate)))
print(np.sum(1/bps))

tidal = '<'
for i in bps:
    tidal += str(i)
    tidal += ' '
tidal += '>'
print(tidal)

previous = 0
count = 1
input_filename = input_filename[:-4]
print("data length: " + str(data.size/samrate) + " " + str(data[0].size))

for i in chops:
    i *= samrate
    chop_index = range(int(previous), int(i))
    chop = [data[index] for index in chop_index]
    chop = np.array(chop)
    output_filename = input_filename + str(count) + '.wav' if count > 9 else input_filename + '0' + str(count) + '.wav'
    print(output_filename + ' length: ' + str(chop.size/samrate))
    wavfile.write(output_filename, samrate, chop)
    count += 1
    previous = i

i = len(data) - 1
chop_index = range(int(previous), int(i))
chop = [data[index] for index in chop_index]
chop = np.array(chop)
output_filename = input_filename + str(count) + '.wav'
wavfile.write(output_filename, samrate, chop)



